### Year 3 Topology 4 Contingency Analysis Reporting - Upper Limit Voltage Violation

In [1]:
import pandas as pd
import numpy as np

#### Bus Voltage Data

In [2]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
list_gen_hydro = [400,500,600] 
for gen in list_gens:
    for lsc in list_lsc:
        for gen_hy in list_gen_hydro:
            file_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'.xlsx'
            data_volt = pd.read_excel(file_out, sheet_name='Bus Voltage')
            data_v_nonan = data_volt.dropna(how='all').reset_index(drop=True)
            data_v_nonan['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_volt.append(data_v_nonan)
data_v = pd.concat(list_volt).reset_index(drop=True)
print(len(data_v))
display(data_v.head())

67720


,BUS,RECORD,TYPE,MIN/DROP,MAX/RISE,CONTINGENCY,BASE VOLTS,CONT VOLTS,DEVIATION,RANGE VIO,DEV VIO,Unnamed: 11,Scenario
0,101 NUC-A 21.600,ALL,RANGE,0.95,1.05,BASE CASE,1.020000,NaN,NaN,NaN,NaN,BASE VOLTS is pu base case voltage.,"Solar = 0 MW, Hydro = 400 MW, LLS"
1,102 NUC-B 21.600,ALL,RANGE,0.95,1.05,BASE CASE,1.020000,NaN,NaN,NaN,NaN,CONT VOLTS is pu contingency case voltage.,"Solar = 0 MW, Hydro = 400 MW, LLS"
2,103 SOLAR_PV 13.800,ALL,RANGE,0.95,1.05,BASE CASE,1.000000,NaN,NaN,NaN,NaN,DEVIATION is difference between contingency ca...,"Solar = 0 MW, Hydro = 400 MW, LLS"
3,151 NUCPANT 500.00,ALL,RANGE,0.95,1.05,BASE CASE,1.013929,NaN,NaN,NaN,NaN,RANGE VIO is range violations calculated as bu...,"Solar = 0 MW, Hydro = 400 MW, LLS"
4,152 MID500 500.00,ALL,RANGE,0.95,1.05,BASE CASE,1.030596,NaN,NaN,NaN,NaN,DEV VIO is deviation violations calculated as ...,"Solar = 0 MW, Hydro = 400 MW, LLS"


#### Bus voltage data wrangling 

In [3]:
data_v = data_v.rename(columns={'BUS':'Bus', 
                                'CONTINGENCY':'Contingency',
                                'BASE VOLTS':'Base Voltage',                               
                                'CONT VOLTS':'Contingency Voltage',
                                'RANGE VIO' : 'Range Violation', 
                                'DEVIATION' : 'Deviation'})
data_v['Contingency'] = data_v['Contingency'].str.replace('&','\&')
data_v['Bus'] = data_v['Bus'].str.replace('_','\_')
data_v['Bus Number'] = data_v['Bus'].str.split(expand=True)[0]

#### Upper limit bus voltage violations

In [4]:
data_high = data_v[data_v['Contingency Voltage']>1.1].reset_index(drop=True)
print(len(data_high))
display(data_high.head())

32


,Bus,RECORD,TYPE,MIN/DROP,MAX/RISE,Contingency,Base Voltage,Contingency Voltage,Deviation,Range Violation,DEV VIO,Unnamed: 11,Scenario,Bus Number
0,211 HYDRO\_G 20.000,ALL,RANGE,0.95,1.05,SING OPN LIN 20 205-206(1),1.038845,1.10168,0.062835,0.05168,NaN,NaN,"Solar = 0 MW, Hydro = 600 MW, LLS",211
1,212 HYDRO\_N 20.000,ALL,RANGE,0.95,1.05,SING OPN LIN 20 205-206(1),1.038845,1.10168,0.062835,0.05168,NaN,NaN,"Solar = 0 MW, Hydro = 600 MW, LLS",212
2,211 HYDRO\_G 20.000,LIMIT,RANGE,0.90,1.10,SING OPN LIN 20 205-206(1),1.038845,1.10168,0.062835,0.00168,NaN,NaN,"Solar = 0 MW, Hydro = 600 MW, LLS",211
3,212 HYDRO\_N 20.000,LIMIT,RANGE,0.90,1.10,SING OPN LIN 20 205-206(1),1.038845,1.10168,0.062835,0.00168,NaN,NaN,"Solar = 0 MW, Hydro = 600 MW, LLS",212
4,211 HYDRO\_G 20.000,ALL,RANGE,0.95,1.05,UNIT 206(1),1.038845,1.10168,0.062835,0.05168,NaN,NaN,"Solar = 0 MW, Hydro = 600 MW, LLS",211


#### Reporting of Buses with upper voltage violations

In [5]:
H_scen = list(data_high['Scenario'].unique())
all_scen = list(data_v['Scenario'].unique())
list_diff = ', '.join(list(set(all_scen) - set(H_scen)))
print(f'It was seen that for Topology 3, {list_diff} did not report any upper voltage limit violation(s).')
list_viol = ', '.join(H_scen)
print(f'For Topology 1, the scenarios, {list_viol} reported upper voltage limit violation(s).')
print('------------------------------------------------------------------------------------------------------------------------------------------------------')
for vhscen in H_scen:
    # Extracting bus data with voltage more than 1.1 PU
    print(vhscen)
    data_hscen = data_high[data_high['Scenario'] == vhscen]
    print(f'For the studied scenario {vhscen}, the buses violating the Upper voltage limits are bus(es)',', '.join(data_hscen['Bus Number'].unique()))
    
    # upper range violation latex table data reporting
    data_rephv = data_hscen[['Bus Number','Contingency','Base Voltage', 'Contingency Voltage', 'Deviation', 'Range Violation']].reset_index(drop=True)
    
    unit_hiv = data_rephv[data_rephv['Contingency'].str.contains('UNIT')]
    if len(unit_hiv) != 0:
        unit_hv = unit_hiv.drop_duplicates(subset=['Bus Number'])
        for conthu in list(unit_hv['Contingency'].unique()):
            unit_cont = unit_hv[unit_hv['Contingency']==conthu]
            busesu = ', '.join(unit_cont['Bus Number'].unique())
            print(f'The buses {busesu} reported violation for the unit fault {conthu}')
            
    
    bus_hiv = data_rephv[data_rephv['Contingency'].str.contains('BUS')]
    if len(bus_hiv) != 0:
        bus_hv = bus_hiv.drop_duplicates(subset=['Bus Number'])
        for conthb in list(bus_hv['Contingency'].unique()):
            bus_cont = bus_hv[bus_hv['Contingency']==conthb]
            busesb = ', '.join(bus_cont['Bus Number'].unique())
            print(f'The buses {busesb} reported violation for the bus fault {conthb}')
            
        
    line_hiv = data_rephv[data_rephv['Contingency'].str.contains('SING OPN LIN')]
    if len(line_hiv) != 0:
        line_hv = line_hiv.drop_duplicates(subset=['Bus Number'])
        for conthl in list(line_hv['Contingency'].unique()):
            line_cont = line_hv[line_hv['Contingency']==conthl]
            busesl = ', '.join(line_cont['Bus Number'].unique())
            print(f'The buses {busesl} reported violation for the single line open fault {conthl}')
    print('------------------------------------------------------------------------------------------------------------------------------------------------------')  

It was seen that for Topology 3, Solar = 50 MW, Hydro = 600 MW, RLS, Solar = 50 MW, Hydro = 500 MW, RLS, Solar = 100 MW, Hydro = 500 MW, LLS, Solar = 50 MW, Hydro = 600 MW, LLS, Solar = 100 MW, Hydro = 400 MW, HLS, Solar = 0 MW, Hydro = 600 MW, RLS, Solar = 100 MW, Hydro = 600 MW, RLS, Solar = 50 MW, Hydro = 500 MW, LLS, Solar = 0 MW, Hydro = 400 MW, HLS, Solar = 50 MW, Hydro = 500 MW, HLS, Solar = 0 MW, Hydro = 500 MW, HLS, Solar = 50 MW, Hydro = 400 MW, HLS, Solar = 0 MW, Hydro = 500 MW, RLS, Solar = 0 MW, Hydro = 400 MW, LLS, Solar = 0 MW, Hydro = 400 MW, RLS, Solar = 100 MW, Hydro = 500 MW, HLS, Solar = 50 MW, Hydro = 400 MW, LLS, Solar = 100 MW, Hydro = 500 MW, RLS, Solar = 0 MW, Hydro = 500 MW, LLS, Solar = 50 MW, Hydro = 400 MW, RLS, Solar = 100 MW, Hydro = 400 MW, LLS did not report any upper voltage limit violation(s).
For Topology 1, the scenarios, Solar = 0 MW, Hydro = 600 MW, LLS, Solar = 0 MW, Hydro = 600 MW, HLS, Solar = 50 MW, Hydro = 600 MW, HLS, Solar = 100 MW, Hydro =

Upper Voltage Limit Counts - Scenario 

In [6]:
vh_index = list(data_high['Scenario'].value_counts().index)
vh_counts = list(data_high['Scenario'].value_counts())
dict_vh_count = {
    'Scenario':vh_index,
    'Violation Counts':vh_counts
}

scen_hv_vc  = pd.DataFrame(dict_vh_count)
scen_hv_vc

,Scenario,Violation Counts
0,"Solar = 0 MW, Hydro = 600 MW, LLS",8
1,"Solar = 100 MW, Hydro = 400 MW, RLS",8
2,"Solar = 0 MW, Hydro = 600 MW, HLS",4
3,"Solar = 50 MW, Hydro = 600 MW, HLS",4
4,"Solar = 100 MW, Hydro = 600 MW, LLS",4
5,"Solar = 100 MW, Hydro = 600 MW, HLS",4


Upper Voltage Limit Counts - Bus

In [7]:
vhb_index = list([i.strip().split()[0] for i in data_high['Bus'].value_counts().index])
vhb_counts = list(data_high['Bus'].value_counts())
dict_vhb_count = {
    'Bus':vhb_index,
    'Violation Counts':vhb_counts
}

bus_hv_vc  = pd.DataFrame(dict_vhb_count)
bus_hv_vc

,Bus,Violation Counts
0,211,16
1,212,16


Upper Voltage Limit Counts - Contingency

In [8]:
vh_index = list(data_high['Contingency'].value_counts().index)
vh_counts = list(data_high['Contingency'].value_counts())
dict_vh_count = {
    'Contingency':vh_index,
    'Violation Counts':vh_counts
}

scen_hv_vc  = pd.DataFrame(dict_vh_count)
scen_hv_vc

,Contingency,Violation Counts
0,SING OPN LIN 20 205-206(1),8
1,UNIT 206(1),8
2,SING OPN LIN 10 201-202(1),8
3,BUS 3005,4
4,BUS 202,4


Upper Voltage Result Summary - grouped by Scenario and Contingency

In [9]:
data_fil_hv = data_high[['Scenario', 'Bus Number', 'Contingency']].drop_duplicates()
pivot_hv = pd.DataFrame(pd.pivot(data_fil_hv, index= ['Scenario','Contingency'], columns = 'Bus Number',values = 'Bus Number').to_records())
pivot_hv['Buses'] = pivot_hv['211'].astype(str).str.cat(pivot_hv[['212']].astype(str), sep=',')
pivot_hv['Buses'] =pivot_hv['Buses'].str.replace(',nan','').str.replace('nan,','')
pivot_high = pivot_hv.drop(columns = list(data_fil_hv['Bus Number'].unique()))

In [10]:
pivot_table_hv = pd.DataFrame(pd.pivot_table(data_fil_hv, index= ['Scenario','Contingency'],values = 'Bus Number', aggfunc='count').to_records())
pivot_table_high = pivot_table_hv.rename(columns = {'Bus Number':'Bus Count'})
df_high = pd.merge(pivot_high, pivot_table_high, on=['Scenario','Contingency'], how='inner').sort_values(by='Bus Count', ascending=False).reset_index(drop=True)
df_high

,Scenario,Contingency,Buses,Bus Count
0,"Solar = 0 MW, Hydro = 600 MW, HLS",BUS 3005,"211,212",2
1,"Solar = 0 MW, Hydro = 600 MW, LLS",SING OPN LIN 20 205-206(1),"211,212",2
2,"Solar = 0 MW, Hydro = 600 MW, LLS",UNIT 206(1),"211,212",2
3,"Solar = 100 MW, Hydro = 400 MW, RLS",SING OPN LIN 20 205-206(1),"211,212",2
4,"Solar = 100 MW, Hydro = 400 MW, RLS",UNIT 206(1),"211,212",2
5,"Solar = 100 MW, Hydro = 600 MW, HLS",SING OPN LIN 10 201-202(1),"211,212",2
6,"Solar = 100 MW, Hydro = 600 MW, LLS",BUS 202,"211,212",2
7,"Solar = 50 MW, Hydro = 600 MW, HLS",SING OPN LIN 10 201-202(1),"211,212",2
